In [2]:
# Import libraries
import pandas as pd
import requests

# Import API keys
from alyssa_config import spoonacular_key, rapidapi_key

In [4]:
# Get list of recipe IDs
recipe_url = "https://api.spoonacular.com/recipes/complexSearch"
recipe_params = {
    'apiKey': spoonacular_key,
    'number': 100}

recipe_response = requests.get(recipe_url, recipe_params).json()
recipe_response

# Output recipe IDs to a csv
recipes_df = pd.DataFrame(recipe_response['results'])
recipes_df.to_csv('Resources/initial_recipes.csv', index=False)

In [6]:
# Import existing recipe IDs
recipes_df = pd.read_csv('Resources/initial_recipes.csv')

In [7]:
recipes_df.head()

,Unnamed: 0,id,title,image,imageType
0,0,782585,Cannellini Bean and Asparagus Salad with Mushr...,https://spoonacular.com/recipeImages/782585-31...,jpg
1,1,716426,"Cauliflower, Brown Rice, and Vegetable Fried Rice",https://spoonacular.com/recipeImages/716426-31...,jpg
2,2,715497,Berry Banana Breakfast Smoothie,https://spoonacular.com/recipeImages/715497-31...,jpg
3,3,715415,Red Lentil Soup with Chicken and Turnips,https://spoonacular.com/recipeImages/715415-31...,jpg
4,4,716406,Asparagus and Pea Soup: Real Convenience Food,https://spoonacular.com/recipeImages/716406-31...,jpg


In [9]:
for id in recipes_df['id']:
    print(id)

782585
716426
715497
715415
716406
644387
715446
782601
795751
766453
716627
664147
640941
715495
794349
716381
660306
756814
715769
642129
642605
636589
646738
715540
663559
662670
640062
715421
648320
716432
798400
782600
652417
639535
729366
661925
794538
649931
636787
641975
652423
715447
716311
715543
157344
658509
716195
716361
765011
660228
716408
661340
782622
715424
659135
639851
664547
633921
658579
715385
641057
655575
632269
637162
982371
1095753
660405
715523
632252
639891
646651
716330
640318
642085
716437
664090
636602
716276
649944
646043
634437
655235
652393
640828
769774
715533
639411
636228
661259
664680
715391
636230
632812
633942
632347
975070
649886
659927
716217
636608


In [10]:
# Get recipe and nutrition information
nutrition_list = []
for id in recipes_df['id']:
    nutrition_url = f"https://api.spoonacular.com/recipes/{id}/information?"
    nutrition_params = {
        'apiKey': spoonacular_key,
        'includeNutrition': "true"}

    nutrition_response = requests.get(nutrition_url, nutrition_params).json()
    nutrition_list.append(nutrition_response)

In [56]:
recipe_name = nutrition_list[0]['title']
recipe_likes = nutrition_list[0]['aggregateLikes']
meal_type = nutrition_list[0]['dishTypes']
cuisines = nutrition_list[0]['cuisines']
num_ingredients = len(nutrition_list[0]['extendedIngredients'])

# Second to last '0' index is for CALORIES
calories_amt = nutrition_list[0]['nutrition']['nutrients'][0]['amount']
calories_unit = nutrition_list[0]['nutrition']['nutrients'][0]['unit']
calories_pct = nutrition_list[0]['nutrition']['nutrients'][0]['percentOfDailyNeeds']

satfat_amt = nutrition_list[0]['nutrition']['nutrients'][2]['amount']
satfat_unit = nutrition_list[0]['nutrition']['nutrients'][2]['unit']
satfat_pct = nutrition_list[0]['nutrition']['nutrients'][2]['percentOfDailyNeeds']

sugar_amt = nutrition_list[0]['nutrition']['nutrients'][5]['amount']
sugar_unit = nutrition_list[0]['nutrition']['nutrients'][5]['unit']
sugar_pct = nutrition_list[0]['nutrition']['nutrients'][5]['percentOfDailyNeeds']

sodium_amt = nutrition_list[0]['nutrition']['nutrients'][7]['amount']
sodium_unit = nutrition_list[0]['nutrition']['nutrients'][7]['unit']
sodium_pct = nutrition_list[0]['nutrition']['nutrients'][7]['percentOfDailyNeeds']

protein_amt = nutrition_list[0]['nutrition']['nutrients'][8]['amount']
protein_unit = nutrition_list[0]['nutrition']['nutrients'][8]['unit']
protein_pct = nutrition_list[0]['nutrition']['nutrients'][8]['percentOfDailyNeeds']